In [1]:
import numpy as np
from scipy import integrate
from matplotlib import pyplot as plt
from matplotlib.legend_handler import HandlerLine2D


'''
NANOG-GATA6-FOXA2-FOXF1 MODEL with separated protein and rna
Model C V1: Add 2 I1-FFLs to regulate FOXF1 and FOXA2 production. I1-FFLs are "pulse generators" that differentially respond to
different production rates.
In this system, FOXF1 is activated at at low GATA6 and inhibited at high GATA6 via a noncompetitive inhibition function,
FOXA2 is activated by GATA6
We observe few differences in dgata6 due to the dominating hill-like equation in the system
'''

def dnanog_rna(nanog, NANOG, GATA6, a1, a2, a3, n, m, KM_NN, KM_GN):
    return (a1 * NANOG ** n)/(KM_NN ** n + NANOG ** n) * (a2 * KM_GN ** m)/(KM_GN ** m + GATA6 ** m) - a3 * nanog

def dgata6_rna(gata6, NANOG, GATA6, dox, p, a1, a2, a3, n, m, KM_GG, KM_NG):
    return dox * p + (a1 * GATA6 ** n)/(KM_GG ** n + GATA6 ** n) * (a2 * KM_NG ** m)/(KM_NG ** m + NANOG ** m) - a3 * gata6

def dy1_rna(y1, GATA6, a0, a1, n, KM_G):
    return (a1 * GATA6 ** n)/(KM_G ** n + GATA6 ** n) - a2 * y1

def dfoxa2_rna(foxa2, GATA6, Y1, a1, a2, a3, KM_G, KM_Y, n, m):
    return (a1 * GATA6 ** n)/(KM_G ** n + GATA6 **n) * (a2 * KM_Y ** m) / (KM_Y ** n + Y1 ** m) - a3 * foxa2

def dY1(Y1, y1, a1, a2):
    return a1 * y1 - a2 * Y1

def dfoxf1_rna(foxf1, GATA6, Y1, a1, a2, a3, KM_G, KM_Y, n, m):
    return (a1 * GATA6 ** n)/(KM_G ** n + GATA6 **n) * (a2 * KM_Y ** m) / (KM_Y ** n + Y1 ** m) - a3 * foxf1

def dFOXF1(FOXF1, foxf1, a1, a2):
    return a1 * foxf1 - a2 * FOXF1

def dNANOG(NANOG, nanog, a1, a2):
    return a1 * nanog - a2 * NANOG

def dGATA6(GATA6, gata6, a1, a2):
    return a1 * gata6 - a2 * GATA6

def dFOXA2(FOXA2, foxa2, a1, a2):
    return a1 * foxa2 - a2 * FOXA2

def dudt(U, t, dox, p, parameters):
    """
    TO DO: Figure out if we should do a reduction of parameters in our system for analysis
    """
    nanog, gata6, y1, foxa2, foxf1, NANOG, GATA6, Y1, FOXA2, FOXF1 = U
    a1, a2, a3, an, am, KM_NN, KM_GN = parameters['nanog_rna']
    b1, b2, b3, bn, bm, KM_GG, KM_NG = parameters['gata6_rna']
    c1, c2, c3, cn, KM_GY = parameters['y1_rna']
    d1, d2, d3, dn, dm, KM_G2, KM_Y2 = parameters['foxa2_rna']
    e1, e2, e3, en, em, KM_G1, KM_Y1 = parameters['foxf1_rna']
    
    y1_translation_rate, y1_degradation_rate = parameters('Y1_prot')
    A2_translation_rate, A2_degradation_rate = parameters('A2_prot')
    A1_translation_rate, A1_degradation_rate = parameters('A1_prot')
    translation_rate, protein_degradation_rate = [0.167, 0.125]
    
    d = 0
    if t > 12:
        d = dox
    dnanog_dt = dnanog_rna(nanog, NANOG, GATA6, a1, a2, a3, an, am, KM_NN, KM_GN)    
    dNANOG_dt = dNANOG(NANOG, nanog, translation_rate, protein_degradation_rate)
    dgata6_dt = dgata6_rna(gata6, NANOG, GATA6, d, p, b1, b2, b3, bn, bm, KM_GG, KM_NG)
    dGATA6_dt = dGATA6(GATA6, gata6, translation_rate, protein_degradation_rate)
    dy1_dt = dy1_rna (y1, GATA6, c1, c2, cn, KM_GY)
    dY1_dt = dY1(Y1, y1, y1_translation_rate, y1_degradation_rate)
    dfoxa2_dt = dfoxa2_rna(foxa2, GATA6, Y1, d1, d2, d3, KM_G2, KM_Y2, dn, dm)
    dFOXA2_dt = dFOXA2(FOXA2, foxa2, A2_translation_rate, A2_protein_degradation_rate)
    dfoxf1_dt = dfoxf1_rna(foxf1, GATA6, Y1, e1, e2, e3, KM_G1, KM_G1, en, em)
    dFOXF1_dt = dFOXF1(FOXF1, foxf1, F1_translation_rate, F1_protein_degradation_rate)
    return dnanog_dt, dgata6_dt, dy1_dt, dfoxa2_dt, dfoxf1_dt, dNANOG_dt, dGATA6_dt, dY1_dt, dFOXA2_dt, dFOXF1_dt


parameters = {}
parameters['nanog_rna'] = []
a0 = 3
b0 = 3
a1 = 4
b1 = 4
a2 = 4
b2 = 4
a3 = 0.5
b3 = 0.5
KM_NN, KM_GN, KM_GG, KM_NG, KM_11, KM_22, KM_12, KM_21 = [5, 5, 5, 5, 5, 5, 5, 5]
KM_G1, KM_G2, KM_GI = [55, 55, 70]
n = m = 4
gata6_n = 8
fox_n = 2

protein_degradation_rate = 0.125
translation_rate = 0.167

SyntaxError: invalid syntax (2386536520.py, line 48)

In [4]:
parameters = {
    'nanog': [0, 1, 2]
}
parameters['nanog']

[0, 1, 2]